In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
loader = TextLoader('Piano Tuning.txt', encoding='utf-8')
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

3570

In [5]:
texts[10]

Document(page_content='***', metadata={'source': 'Piano Tuning.txt'})

In [6]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 9.55 s
Wall time: 15.6 s


In [9]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("Piano Tuning.txt')")

In [10]:
len(docs)

2

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [13]:
query = "What is the steps of piano tunning?"
process_llm_response(qa_chain, query)

Query: What is the steps of piano tunning?

Inference time: 5.258 sec.

Result:  The steps of piano tuning involve several processes, including preparing the piano, checking the tuning stability, adjusting the pitch, and fine-tuning the strings. The tuner uses a tuning hammer to gently tap the strings and listen for the correct pitch, making small adjustments as needed. The process can take several hours for a large piano, and it requires a skilled and experienced tuner.

metadata: {'source': 'Piano Tuning.txt'}


In [14]:
query = "What is the general construction of the piano?"
process_llm_response(qa_chain, query)

Query: What is the general construction of the piano?

Inference time: 5.069 sec.

Result:  The piano is a musical instrument that consists of a wooden frame, strings, hammers, keys, and pedals. The strings are stretched across the frame and are struck by the hammers when the keys are pressed. The sound produced by the strings is amplified by a soundboard, which is also part of the frame. The pedals allow the player to sustain or dampen the sound of the notes. The overall design of the piano has evolved over time, with improvements in materials and technology leading to more refined and sophisticated instruments.

metadata: {'source': 'Piano Tuning.txt'}


In [15]:
query = "Name all the defects to which the key is subject"
process_llm_response(qa_chain, query)
#wrong metadata

Query: Name all the defects to which the key is subject

Inference time: 3.435 sec.

Result:  Some possible defects that the key could be subject to include incompleteness, inaccuracy, and potentially others listed in the contract or agreement. Without further context, it's impossible to say for certain what other defects might be specified, so it's best to simply list the ones provided.

metadata: {'source': 'Piano Tuning.txt'}


In [16]:
query = "Give 1 Technical Names and Uses of the Parts of the Upright Action? "
process_llm_response(qa_chain, query)

Query: Give 1 Technical Names and Uses of the Parts of the Upright Action? 

Inference time: 3.856 sec.

Result:  The hammer is a component of the action in a piano that strikes the strings when a key is pressed. Its technical name is the "hammer." The hammer is made of felt and is attached to a wooden frame called the "shank." When a key is pressed, the hammer is propelled towards the strings by a spring-loaded mechanism called the "jack." The hammer strikes the strings, causing them to vibrate and produce sound.

metadata: {'source': 'Piano Tuning.txt'}


In [17]:
query = "How to repair faults in a piano aside from the action?"
process_llm_response(qa_chain, query)

Query: How to repair faults in a piano aside from the action?

Inference time: 0.328 sec.

Result:  Some common faults in pianos that are not related to the action (the mechanism that causes the keys to move) include:

1. Loose or missing keys: If a key is loose or missing, it can be replaced or repaired by a piano technician. The technician will remove the old key and replace it with a new one that matches the color and texture of the surrounding keys.

2. Cracked or warped soundboard: The soundboard is the large wooden board that vibrates when the strings are struck. If it becomes cracked or warped, it can affect the tone and volume of the piano. A technician can repair or replace the soundboard to restore the piano's sound.

3. Damaged or missing strings: If a string breaks, it can be replaced by a technician. If multiple strings are broken, it may be a sign of a more serious issue, such as a weak or damaged tuning pin block.

4. Damaged or missing tuning pins: The tuning pins are t

In [18]:
query = "1 example of Technique or Modus Operandi in Piano Tuning"
process_llm_response(qa_chain, query)

Query: 1 example of Technique or Modus Operandi in Piano Tuning

Inference time: 6.148 sec.

Result:  In piano tuning, one example of technique or modus operandi is the use of a tuning hammer to gently tap on the strings to determine their pitch and adjust them to the correct frequency. This technique involves striking the string with the hammer at a specific angle and force to produce a clear and accurate tone. The tuner also listens carefully to the sound of the string and makes small adjustments to the tension of the string to bring it into harmony with the other notes in the piano. This process is repeated for each note on the keyboard, ensuring that the entire piano is in tune and producing a rich and harmonious sound.

metadata: {'source': 'Piano Tuning.txt'}


In [19]:
query = "How to clean the piano?"
process_llm_response(qa_chain, query)
#wrong metadata

Query: How to clean the piano?

Inference time: 4.716 sec.

Result:  To clean a piano, you can use a soft-bristled brush to remove any loose dust from the keys, followed by a slightly damp cloth to gently wipe away any remaining dirt or grime. Be sure to avoid getting water on the wooden parts of the piano, as this can cause damage over time. It's also a good idea to have a professional piano technician perform regular maintenance and tuning to keep the instrument in top condition.

metadata: {'source': 'The Great Gatsyby.txt'}


In [20]:
query = "How to fix if there is a leak is found in the air boards?"
process_llm_response(qa_chain, query)

Query: How to fix if there is a leak is found in the air boards?

Inference time: 3.219 sec.

Result:  If a leak is found in the air boards, such as a crack or split, it can be stopped permanently by gluing a piece of bellows cloth or any good quality adhesive tape over the affected area. This will prevent air from escaping and ensure proper functioning of the air boards.

metadata: {'source': 'Piano Tuning.txt'}


In [21]:
query = "When a key snaps or clicks at the instant it is let up, give two conditions that might cause it"
process_llm_response(qa_chain, query)
#wrong metadata

Query: When a key snaps or clicks at the instant it is let up, give two conditions that might cause it

Inference time: 3.485 sec.

Result:  1. Dirt or debris may be lodged between the key and the tumbler, causing a snapping or clicking sound as the key is lifted. 2. The tumbler pins may be worn or misaligned, causing them to bind or catch as the key is lifted, resulting in a snapping or clicking sound.

metadata: {'source': 'Piano Tuning.txt'}


In [22]:
query = "Give two causes for defective damping in a square piano"
process_llm_response(qa_chain, query)

Query: Give two causes for defective damping in a square piano

Inference time: 6.544 sec.

Result:  One cause of defective damping in a square piano could be worn or damaged dampers. Over time, the felt on the dampers can become compressed, causing them to lose their ability to effectively mute the strings. This can result in a persistent ringing or sustain of the notes, even after they have been released. Another possible cause is a lack of tension in the damping mechanism itself. This can occur if the strings become too loose or if the dampers are not properly secured to the action. In either case, the result is a loss of control over the decay of the sound, making it difficult to play with precision and expression.

metadata: {'source': 'Piano Tuning.txt'}


In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x2cb90af1b90>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
